In [2]:
import newspaper
from newspaper import Article
from mongoengine import *
from gensim.models.keyedvectors import KeyedVectors
from DocSim import DocSim
import newspaper
import nltk
import gensim
connect('news-app')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

In [ ]:
class NewsArticle(Document):
    title = StringField()
    image_url = StringField()
    source = StringField()
    summary = StringField()
    text = StringField()
    link = StringField()
    keywords = ListField(StringField())
    author = StringField()
    publish_date = DateField()
    truth_value = IntField()

class ArticleCollection(Document):
    summary = StringField()
    articles = ListField(ReferenceField(NewsArticle))
    selected_article = ReferenceField(NewsArticle)
        
        
class Articles(Document):
    title = StringField()
    truth_value = IntField()
    full_article = ReferenceField(ArticleCollection)
    is_featured = BooleanField()
    

In [20]:
def find_article(text, articles):
    for article in articles:
        if article.text == text:
            return article

model_path = './GoogleNews-vectors-negative300.bin'
w2v_model = KeyedVectors.load_word2vec_format(model_path, binary=True)

ds = DocSim(w2v_model)

print("Model loaded.")

Model loaded.


In [ ]:
sources = ['https://www.theverge.com','https://pitchfork.com','https://slate.com']
built_sources = []
for source in sources:
    built_sources.append(newspaper.build(source, memoize_articles=False))

print("Sources built.")
inserted_titles = []
first_time = True
if len(ArticleCollection.objects):
    first_time = False
for source in built_sources:
    for article in source.articles:
        if article.title not in inserted_titles:
            
            article.download()
            article.parse()
            article.nlp()
            title = article.title
            inserted_titles.append(title)
            image_url = article.top_image
            source = article.source_url
            summary = article.summary
            text = article.text
            link = article.url
            keywords = article.keywords
            author = ', '.join(article.authors)
            publish_date = article.publish_date
            truth_value = 50
            print("Adding - "+title)
            if not first_time:
                source = text
                targets = [i.text for i in NewsArticle.objects]
                sim_scores = ds.calculate_similarity(source, targets)
                print('Not first -----')
                if not sim_scores or not len(sim_scores):
                    print('Skipped Unsupported - ' + title)
                    continue
                if sim_scores[0]['score']<0.99 and sim_scores[0]['score']>=0.92:
                    similar_article = find_article(sim_scores[0]['doc'], NewsArticle.objects)
                    art = NewsArticle(title=title,
                              image_url=image_url,
                              source=source,
                              summary=summary,
                              text=text,
                              link=link,
                              keywords=keywords,
                              author=author,
                              publish_date=publish_date,
                              truth_value=truth_value
                             )
                    art.save()
                    collection=ArticleCollection.objects(articles__contains=similar_article)[0]
                    collection.articles.append(art.to_dbref())
                    collection.save()
                    print('Saved to existing collection - ' + title)
                elif sim_scores[0]['score']<0.92:
                    art = NewsArticle(title=title,
                              image_url=image_url,
                              source=source,
                              summary=summary,
                              text=text,
                              link=link,
                              keywords=keywords,
                              author=author,
                              publish_date=publish_date,
                              truth_value=truth_value
                             )
                    art.save()
                    collection=ArticleCollection(selected_article=art.to_dbref(),summary=art.summary, articles=[art.to_dbref()])
                    collection.save()
                    print('Saved to new collection - ' + title)
                else:
                    print('Skipped - ' + title)
                    continue
            else:
                art = NewsArticle(title=title,
                              image_url=image_url,
                              source=source,
                              summary=summary,
                              text=text,
                              link=link,
                              keywords=keywords,
                              author=author,
                              publish_date=publish_date,
                              truth_value=truth_value
                             )
                art.save()
                collection=ArticleCollection(selected_article=art.to_dbref(),summary=art.summary, articles=[art.to_dbref()])
                collection.save()
                first_time = False
                print('Saved to new collection as first time - ' + title)

Sources built.
Adding - Magic Keyboard for the iPad Pro review: the best way to turn an iPad into a laptop
Not first -----
Saved to new collection - Magic Keyboard for the iPad Pro review: the best way to turn an iPad into a laptop
Adding - The OnePlus 8 series is much cheaper in India than the US
Not first -----
Saved to new collection - The OnePlus 8 series is much cheaper in India than the US
Adding - Apple’s bigger iPad Pro with Magic Keyboard weighs more than a MacBook Air
Not first -----
Saved to existing collection - Apple’s bigger iPad Pro with Magic Keyboard weighs more than a MacBook Air
Adding - LG teases new 5G Velvet phone with ‘raindrop’ camera
Not first -----
Saved to new collection - LG teases new 5G Velvet phone with ‘raindrop’ camera
Adding - Facebook will expand its symptom tracking survey globally to measure the spread of COVID-19
Not first -----
Saved to existing collection - Facebook will expand its symptom tracking survey globally to measure the spread of COVID-1

Adding - Samsung’s smartwatches get a hand-washing reminder and timer app
Not first -----
Saved to new collection - Samsung’s smartwatches get a hand-washing reminder and timer app
Adding - Samsung’s smartwatches get a hand-washing reminder and timer app
Not first -----
Skipped - Samsung’s smartwatches get a hand-washing reminder and timer app
Adding - Amazon gets approval for UK food delivery investment as pandemic hammers industry
Not first -----
Saved to existing collection - Amazon gets approval for UK food delivery investment as pandemic hammers industry
Adding - Amazon gets approval for UK food delivery investment as pandemic hammers industry
Not first -----
Skipped - Amazon gets approval for UK food delivery investment as pandemic hammers industry
Adding - A video uses lasers to show how you spit when you talk
Not first -----
Saved to existing collection - A video uses lasers to show how you spit when you talk
Adding - Mrs. America shows why the implosion of the Equal Rights Ame

Adding - There’s a new label to vet brands’ climate change pledges
Not first -----
Saved to existing collection - There’s a new label to vet brands’ climate change pledges
Adding - This little self-driving boat is changing the way we search for shipwrecks
Not first -----
Saved to existing collection - This little self-driving boat is changing the way we search for shipwrecks
Adding - This little self-driving boat is changing the way we search for shipwrecks
Not first -----
Skipped - This little self-driving boat is changing the way we search for shipwrecks
Adding - Animals might help us better understand the long-term health effects of wildfires
Not first -----
Saved to existing collection - Animals might help us better understand the long-term health effects of wildfires
Adding - Birth control and books can slow down climate change
Not first -----
Saved to existing collection - Birth control and books can slow down climate change
Adding - Maps show drastic drop in China’s air pollutio

Adding - The best graphs and data for tracking the coronavirus pandemic
Not first -----
Skipped - The best graphs and data for tracking the coronavirus pandemic
Adding - How to use Slack channels
Not first -----
Saved to existing collection - How to use Slack channels
Adding - The best alternatives to Zoom for videoconferencing
Not first -----
Saved to existing collection - The best alternatives to Zoom for videoconferencing
Adding - Six Dark Sky alternatives for Android weather watchers
Not first -----
Saved to existing collection - Six Dark Sky alternatives for Android weather watchers
